# NN

In [3]:
import netCDF4

import numpy as np

f = netCDF4.Dataset('D:\\sst.mnmean.nc')  # ftp://ftp.cdc.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc
SST = f.variables['sst'][-1203:, :, :].data

In [4]:
loc = (134, 44)  # 134, 44 表示（134 * 2）°E→92°W, （88 - 44 * 2）°N→0°

In [5]:
A = SST > -2  # 1表示海洋，0表示陆地
A = np.average(A, axis = 0)

In [6]:
new_SST = np.zeros((len(SST), int(np.sum(A))))  # 合并SST数据从一个三维数组到一个二维数组

s = 0

for i in range(89):
    for j in range(180):
        if A[i, j] == 1:
            if (j, i) == loc:
                print(j, i, ':', s)
            new_SST[:, s] += SST[:, i, j]
            s += 1

134 44 : 5255


In [7]:
month_mean = np.zeros((12, int(np.sum(A))))
month_s = np.arange(0, 1200, 12)
month_s

array([   0,   12,   24,   36,   48,   60,   72,   84,   96,  108,  120,
        132,  144,  156,  168,  180,  192,  204,  216,  228,  240,  252,
        264,  276,  288,  300,  312,  324,  336,  348,  360,  372,  384,
        396,  408,  420,  432,  444,  456,  468,  480,  492,  504,  516,
        528,  540,  552,  564,  576,  588,  600,  612,  624,  636,  648,
        660,  672,  684,  696,  708,  720,  732,  744,  756,  768,  780,
        792,  804,  816,  828,  840,  852,  864,  876,  888,  900,  912,
        924,  936,  948,  960,  972,  984,  996, 1008, 1020, 1032, 1044,
       1056, 1068, 1080, 1092, 1104, 1116, 1128, 1140, 1152, 1164, 1176,
       1188])

In [8]:
for i in range(12):
    month_mean[i] = np.copy(np.average(new_SST[month_s[:95] + i, :], axis = 0))
month_mean[:, 5255]  # 波罗的海某处海域1919年至2014年95年月平均海温

array([24.84811632, 26.15498543, 26.67061384, 26.13181588, 25.0056647 ,
       23.89235374, 22.99094383, 22.40314257, 22.35578517, 22.64520766,
       22.97098622, 23.55253011])

In [9]:
for i in range(12):
    new_SST[month_s + i, :] -= month_mean[i]
for i in range(3):
    new_SST[1200 + i, :] -= month_mean[i]

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)  # 设置PCA想要降到的维数

pca.fit(new_SST[:1140])

np.sum(pca.explained_variance_ratio_)

0.9203976374519192

In [11]:
X = new_SST @ pca.components_.T

X_std = np.std(X[:1140], axis = 0)
X /= X_std

In [10]:
X_train = X[:1139]
X_valid = X[1139:1199]
X_test = X[-2]

In [11]:
y_train = new_SST[1:1140, 5255]
y_valid = new_SST[1140:1200, 5255]

In [12]:
import torch

X_train = torch.from_numpy(X_train).float()
X_valid = torch.from_numpy(X_valid).float()
X_test = torch.from_numpy(X_test.reshape(1, 100)).float()
y_train = torch.from_numpy(y_train.reshape(len(y_train), 1)).float()
y_valid = torch.from_numpy(y_valid.reshape(len(y_valid), 1)).float()

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from adabound import AdaBound

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(100, 3)
        self.fc2 = nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
net = Net()

criterion = nn.MSELoss()
optimizer = AdaBound(net.parameters(), lr=0.001)

In [15]:
for epoch in range(10000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        outputs = net(X_valid)
        loss_valid = criterion(outputs, y_valid)
        print(epoch + 1, loss.item(), loss_valid.item())
        
print('Finished Training')

1 1.0736862421035767 1.9411107301712036
1001 0.11922675371170044 0.5181803107261658
2001 0.11572258919477463 0.4422978162765503
3001 0.11484316736459732 0.4524969756603241
4001 0.11483649164438248 0.4530077576637268
5001 0.11483722925186157 0.4530792236328125
6001 0.11483823508024216 0.452997624874115
7001 0.11483677476644516 0.4530576467514038
8001 0.1148369088768959 0.4529978632926941
9001 0.11483655124902725 0.45295655727386475
Finished Training


In [16]:
outputs = net(X_valid)
loss = criterion(outputs, y_valid)

for i in range(60):
    print(y_valid[i].item() + month_mean[i % 12, 5255], outputs[i].item() + month_mean[i % 12, 5255])
    
print('\n')

print(loss.item())

25.193620695879584 25.375490709355002
25.725605010986328 26.61388871073723
26.790624619314546 26.78163404841172
26.549350738211682 26.296852737351468
26.679452894863328 25.08423972004338
26.143993345059847 24.70031222418735
24.624321022786592 24.6988294513602
23.667221065571432 24.155559774448996
23.38800243327492 23.606899895166094
23.729892743261235 23.508764637143987
24.111202208619368 24.06691154680754
24.508861539865794 24.04925975015289
25.268852248003608 24.726700275352126
26.084409713745117 26.083786517381668
27.0227508403753 26.367520079800958
27.623594253941587 26.342915892287305
27.091974376377305 26.014590262111863
26.963388410367465 25.941001382626986
26.370748439588045 25.264405885495638
24.83215152840865 25.128507967999106
25.315359153245623 24.015954651330645
25.57662965122022 24.819972050817388
26.050239650826704 25.486461011986982
26.717212734724345 25.514400897527995
27.22409240823043 26.763605400135642
27.63623046875 27.909051537513733
28.250402406642312 27.84172292

预测（92°W，0°）今年3月份海温（真实值为27.26599℃）

In [17]:
y_test = net(X_test) + month_mean[2, 5255]
y_test.item()

27.77191925048828

使用连续六个月预测第七个月的海温，依旧使用NN

In [18]:
seq = 6

X_train = np.zeros((1140 - seq, seq * 100), dtype = np.float32)
X_valid = np.zeros((60, seq * 100), dtype = np.float32)
X_test = np.zeros((1, seq * 100), dtype = np.float32)

for i in range(6):
    X_train[:, i * 100:i * 100 + 100] = X[i:1140 - seq + i]
    X_valid[:, i * 100:i * 100 + 100] = X[1140 - seq + i:1200 - seq + i]
    X_test[:, i * 100:i * 100 + 100] = X[- (seq + 1) + i]

y_train = new_SST[seq:1140, 5255]
y_valid = new_SST[1140:1200, 5255]

import torch

X_train = torch.from_numpy(X_train).float()
X_valid = torch.from_numpy(X_valid).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train.reshape(len(y_train), 1)).float()
y_valid = torch.from_numpy(y_valid.reshape(len(y_valid), 1)).float()

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from adabound import AdaBound

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(600, 3)
        self.fc2 = nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [21]:
net = Net()

criterion = nn.MSELoss()
optimizer = AdaBound(net.parameters(), lr=0.001)

In [22]:
for epoch in range(10000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        outputs = net(X_valid)
        loss_valid = criterion(outputs, y_valid)
        print(epoch + 1, loss.item(), loss_valid.item())
        
print('Finished Training')

1 1.6621829271316528 3.464352607727051
1001 0.14149276912212372 1.886434555053711
2001 0.14045922458171844 1.882771372795105
3001 0.14045710861682892 1.8813800811767578
4001 0.1404571384191513 1.8810398578643799
5001 0.1404571384191513 1.880966305732727
6001 0.14045710861682892 1.8809258937835693
7001 0.1404571235179901 1.880924940109253
8001 0.1404571384191513 1.8809261322021484
9001 0.14045709371566772 1.8809269666671753
Finished Training


In [23]:
outputs = net(X_valid)
loss = criterion(outputs, y_valid)

for i in range(60):
    print(y_valid[i].item() + month_mean[i % 12, 5255], outputs[i].item() + month_mean[i % 12, 5255])
    
print('\n')

print(loss.item())

25.193620695879584 26.21408490758193
25.725605010986328 27.24207615852356
26.790624619314546 27.251985744426126
26.549350738211682 25.954872578068784
26.679452894863328 25.64826047295018
26.143993345059847 23.74592682436893
24.624321022786592 23.87535569981525
23.667221065571432 23.536782618572836
23.38800243327492 23.065786160920794
23.729892743261235 22.386827839048284
24.111202208619368 24.127846805672895
24.508861539865794 25.03056293537742
25.268852248003608 26.38736371617568
26.084409713745117 26.498181343078613
27.0227508403753 26.509788350055093
27.623594253941587 25.57552430002313
27.091974376377305 25.5663887249796
26.963388410367465 24.552252081193423
26.370748439588045 23.856938639440035
24.83215152840865 23.495555635502463
25.315359153245623 23.812286295388873
25.57662965122022 24.251610291631597
26.050239650826704 24.284673182587873
26.717212734724345 24.731226978803935
27.22409240823043 26.45451895814193
27.63623046875 27.761388063430786
28.250402406642312 27.05503983874

预测（92°W，0°）今年3月份海温（真实值为27.26599℃）

In [24]:
y_test = net(X_test) + month_mean[2, 5255]
y_test.item()

27.486730575561523

# RNN

In [36]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

class Rnn(nn.Module):

    def __init__(self):
        super(Rnn, self).__init__()
        self.rnn = nn.RNN(
            input_size=100,
            hidden_size=2,
            num_layers=1,
            batch_first=True
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x, _ = self.rnn(x, None)
        x = self.fc(x[:, -1, :])  # 只计算最后一个步长的输出
        return x

In [36]:
seq = 6

X_train = np.zeros((1140 - seq, seq, 100), dtype = np.float32)
X_valid = np.zeros((60, seq, 100), dtype = np.float32)
X_test = np.zeros((1, seq, 100), dtype = np.float32)

for i in range(seq):
    X_train[:, i] = X[i:1140 - seq + i]
    X_valid[:, i] = X[1140 - seq + i:1200 - seq + i]
    X_test[:, i] = X[- (seq + 1) + i]

y_train = new_SST[seq:1140, 5255]
y_valid = new_SST[1140:1200, 5255]

import torch

X_train = torch.from_numpy(X_train).float()
X_valid = torch.from_numpy(X_valid).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train.reshape(len(y_train), 1)).float()
y_valid = torch.from_numpy(y_valid.reshape(len(y_valid), 1)).float()

In [37]:
net = Rnn()

criterion = nn.MSELoss()
optimizer = AdaBound(net.parameters(), lr=0.0001)

In [38]:
for epoch in range(50000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        outputs = net(X_valid)
        loss_valid = criterion(outputs, y_valid)
        print(epoch + 1, loss.item(), loss_valid.item())
        
print('Finished Training')

1 1.8713352680206299 3.143311023712158
1001 0.827667236328125 1.3726789951324463
2001 0.36556875705718994 0.6758120656013489
3001 0.20847921073436737 0.40724092721939087
4001 0.16274335980415344 0.33378124237060547
5001 0.15482378005981445 0.3390403091907501
6001 0.1502528041601181 0.33562931418418884
7001 0.14719653129577637 0.33005768060684204
8001 0.14609143137931824 0.32994475960731506
9001 0.1455148458480835 0.3327111601829529
10001 0.14452959597110748 0.33963245153427124
11001 0.14340758323669434 0.34315335750579834
12001 0.14204926788806915 0.3454020917415619
13001 0.14066240191459656 0.3488912284374237
14001 0.1402275562286377 0.3485538363456726
15001 0.13993574678897858 0.3476327657699585
16001 0.13963928818702698 0.3467995226383209
17001 0.13899187743663788 0.34561482071876526
18001 0.13852889835834503 0.34731075167655945
19001 0.13829493522644043 0.34728753566741943
20001 0.13810822367668152 0.3471439480781555
21001 0.13794583082199097 0.34722191095352173
22001 0.13779971003

In [41]:
outputs = net(X_valid)
loss = criterion(outputs, y_valid)

for i in range(60):
    print(y_valid[i].item() + month_mean[i % 12, 5255], outputs[i].item() + month_mean[i % 12, 5255])
    
print('\n')

print(loss.item())

25.193620695879584 24.971479340603477
25.725605010986328 26.54998779296875
26.790624619314546 25.9322690524553
26.549350738211682 26.103351801320127
26.679452894863328 25.29735934608861
26.143993345059847 25.654691544332003
24.624321022786592 24.72461000994632
23.667221065571432 23.92221295457137
23.38800243327492 23.459651627038653
23.729892743261235 23.71031225982465
24.111202208619368 24.258329717736494
24.508861539865794 24.540400145555797
25.268852248003608 26.202216311504966
26.084409713745117 26.252562880516052
27.0227508403753 26.501388148257607
27.623594253941587 26.47641679613214
27.091974376377305 26.570494531330308
26.963388410367465 26.06452247218082
26.370748439588045 25.877356448926424
24.83215152840865 25.680879231503134
25.315359153245623 24.052516974900897
25.57662965122022 25.544136775167363
26.050239650826704 25.870408861260664
26.717212734724345 26.72887164166099
27.22409240823043 28.03124229531539
27.63623046875 28.05290412902832
28.250402406642312 27.822111205050

预测（92°W，0°）今年3月份海温（真实值为27.26599℃）

In [34]:
y_test = net(X_test) + month_mean[2, 5255]
y_test.item()

27.55971908569336

修改RNN，将损失函数的计算方案改为对RNN的全部输出做计算

In [42]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

class Rnn(nn.Module):

    def __init__(self):
        super(Rnn, self).__init__()
        self.rnn = nn.RNN(
            input_size=100,
            hidden_size=2,
            num_layers=1,
            batch_first=True
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x, _ = self.rnn(x, None)
        outs = []
        for time_step in range(x.size(1)):
            outs.append(self.fc(x[:, time_step, :]))  # 计算6个步长全部的输出
        return torch.stack(outs, dim=1)

In [43]:
seq = 6

X_train = np.zeros((1140 - seq, seq, 100), dtype = np.float32)
X_valid = np.zeros((60, seq, 100), dtype = np.float32)
X_test = np.zeros((1, seq, 100), dtype = np.float32)

for i in range(seq):
    X_train[:, i] = X[i:1140 - seq + i]
    X_valid[:, i] = X[1140 - seq + i:1200 - seq + i]
    X_test[:, i] = X[- (seq + 1) + i]

y_train = np.zeros((1140 - seq, seq, 1), dtype = np.float32)
y_valid = np.zeros((60, seq, 1), dtype = np.float32)   

for i in range(seq):
    y_train[:, i, 0] = new_SST[i + 1:1140 - seq + i + 1, 5255]
    y_valid[:, i, 0] = new_SST[1140 - seq + i + 1:1200 - seq + i + 1, 5255]

import torch

X_train = torch.from_numpy(X_train).float()
X_valid = torch.from_numpy(X_valid).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_valid = torch.from_numpy(y_valid).float()

In [44]:
net = Rnn()

criterion = nn.MSELoss()
optimizer = AdaBound(net.parameters(), lr=0.001)

In [45]:
for epoch in range(50000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        outputs = net(X_valid)
        loss_valid = criterion(outputs, y_valid)
        print(epoch + 1, loss.item(), loss_valid.item())
        
print('Finished Training')

1 1.5482722520828247 2.123746633529663
1001 0.38158518075942993 1.086923599243164
2001 0.1753682643175125 0.35406428575515747
3001 0.16112695634365082 0.3336654305458069
4001 0.15865999460220337 0.3256247341632843
5001 0.15659010410308838 0.32952776551246643
6001 0.15511101484298706 0.3353109657764435
7001 0.1533314287662506 0.3444243371486664
8001 0.15203750133514404 0.35505107045173645
9001 0.1501304805278778 0.36284497380256653
10001 0.14895951747894287 0.3647443950176239
11001 0.14830711483955383 0.36568397283554077
12001 0.1473568230867386 0.3679659068584442
13001 0.1459105759859085 0.3682037889957428
14001 0.1454010307788849 0.3651854693889618
15001 0.1451156884431839 0.36486977338790894
16001 0.14492495357990265 0.36540281772613525
17001 0.14475223422050476 0.36573970317840576
18001 0.14453986287117004 0.3655436635017395
19001 0.14430946111679077 0.3651895225048065
20001 0.14410404860973358 0.3649638593196869
21001 0.14391417801380157 0.3648534119129181
22001 0.14375931024551392

In [46]:
outputs = net(X_valid)
loss = criterion(outputs, y_valid)

for i in range(60):
    print(y_valid[i][-1].item() + month_mean[i % 12, 5255], outputs[i][-1].item() + month_mean[i % 12, 5255])
    
print('\n')

print(loss.item())

25.193620695879584 24.970108314564353
25.725605010986328 26.76147896051407
26.790624619314546 26.21222157854783
26.549350738211682 25.907193749829343
26.679452894863328 25.637078820404252
26.143993345059847 25.134251561917758
24.624321022786592 25.06894914225528
23.667221065571432 23.574892993977194
23.38800243327492 23.335128941034014
23.729892743261235 24.219134700925725
24.111202208619368 24.63671669207121
24.508861539865794 24.64641064217216
25.268852248003608 25.50362019162429
26.084409713745117 26.523492455482483
27.0227508403753 26.14056654352891
27.623594253941587 26.231266349240354
27.091974376377305 26.32572460049077
26.963388410367465 25.928559270657992
26.370748439588045 25.505715766705965
24.83215152840865 25.44663536172164
25.315359153245623 24.83968619296425
25.57662965122022 25.359886897237676
26.050239650826704 25.8259395047238
26.717212734724345 26.219675121809306
27.22409240823043 27.62012807946456
27.63623046875 28.200563430786133
28.250402406642312 27.8730996407960

预测（92°W，0°）今年3月份海温（真实值为27.26599℃）

In [51]:
y_test = net(X_test)[-1, -1] + month_mean[2, 5255]
y_test.item()

27.78188705444336

# LSTM

In [27]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

class lstm(nn.Module):

    def __init__(self):
        super(lstm, self).__init__()
        self.rnn = nn.LSTM(
            input_size=100,
            hidden_size=2,
            num_layers=1,
            batch_first=True
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x, _ = self.rnn(x, None)
        x = self.fc(x[:, -1, :])
        return x

In [28]:
net = lstm()

criterion = nn.MSELoss()
optimizer = AdaBound(net.parameters(), lr=0.0001)

In [12]:
seq = 6

X_train = np.zeros((1140 - seq, seq, 100), dtype = np.float32)
X_valid = np.zeros((60, seq, 100), dtype = np.float32)
X_test = np.zeros((1, seq, 100), dtype = np.float32)

for i in range(seq):
    X_train[:, i] = X[i:1140 - seq + i]
    X_valid[:, i] = X[1140 - seq + i:1200 - seq + i]
    X_test[:, i] = X[- (seq + 1) + i]

y_train = new_SST[seq:1140, 5255]
y_valid = new_SST[1140:1200, 5255]

import torch

X_train = torch.from_numpy(X_train).float()
X_valid = torch.from_numpy(X_valid).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train.reshape(len(y_train), 1)).float()
y_valid = torch.from_numpy(y_valid.reshape(len(y_valid), 1)).float()

In [29]:
for epoch in range(50000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        outputs = net(X_valid)
        loss_valid = criterion(outputs, y_valid)
        print(epoch + 1, loss.item(), loss_valid.item())
        
print('Finished Training')

1 1.3009282350540161 1.446370244026184
1001 0.9506367444992065 1.7115427255630493
2001 0.7413881421089172 1.796886682510376
3001 0.5693593621253967 1.839682698249817
4001 0.43132254481315613 1.6995794773101807
5001 0.3307509422302246 1.6118377447128296
6001 0.24793072044849396 1.6019963026046753
7001 0.19230233132839203 1.6059205532073975
8001 0.14692556858062744 1.5775210857391357
9001 0.11521155387163162 1.6153767108917236
10001 0.09273379296064377 1.3916842937469482
11001 0.07819138467311859 1.531123399734497
12001 0.06656265258789062 1.5809093713760376
13001 0.055427614599466324 1.5266815423965454
14001 0.0465288944542408 1.5292026996612549
15001 0.04102608188986778 1.6938410997390747
16001 0.03762657940387726 1.7657458782196045
17001 0.0353761650621891 1.8386359214782715
18001 0.03308688476681709 1.9795353412628174
19001 0.03135392442345619 2.11124849319458
20001 0.030026959255337715 2.14719557762146
21001 0.028564903885126114 2.2290260791778564
22001 0.026529015973210335 2.229907

In [17]:
outputs = net(X_valid)
loss = criterion(outputs, y_valid)

for i in range(60):
    print(y_valid[i].item() + month_mean[i % 12, 5255], outputs[i].item() + month_mean[i % 12, 5255])
    
print('\n')

print(loss.item())

25.193620695879584 25.47861276488555
25.725605010986328 25.96190918982029
26.790624619314546 26.4775351740812
26.549350738211682 25.8547396656714
26.679452894863328 26.21907484405919
26.143993345059847 24.86085501030872
24.624321022786592 23.394879439630007
23.667221065571432 22.605400364806776
23.38800243327492 23.37576905677193
23.729892743261235 20.877672565610784
24.111202208619368 21.432042686562788
24.508861539865794 23.22151946833259
25.268852248003608 25.983844085743552
26.084409713745117 27.135578334331512
27.0227508403753 28.02400453467118
27.623594253941587 26.013001128717473
27.091974376377305 26.54725813740178
26.963388410367465 25.434598413266635
26.370748439588045 22.973363408603166
24.83215152840865 23.55512177567733
25.315359153245623 23.735083856080706
25.57662965122022 23.265480590494054
26.050239650826704 23.246814457993757
26.717212734724345 23.358791334534946
27.22409240823043 24.662781729509955
27.63623046875 25.693006098270416
28.250402406642312 26.2789337791894

查看参数

In [19]:
params = net.state_dict()
for key, value in params.items():
    print(key)
    print(value)

rnn.weight_ih_l0
tensor([[-1.6852, -0.6216, -0.8943, -0.2422,  1.7275, -0.2259,  0.9207,
         -0.3709,  2.8312,  0.6466, -2.8526,  1.6388, -0.2912, -0.0726,
         -2.4687, -0.6574,  0.6060,  2.4428, -3.6113, -1.0191, -2.9080,
         -2.9458, -0.1578,  2.8558, -0.6879, -0.1257,  2.2011,  1.1495,
         -0.6167, -0.8097, -1.4896, -1.6527,  3.1448, -0.9198,  0.3121,
          2.5216,  0.4371, -1.7363, -1.8105, -0.9995, -1.3106,  1.7906,
         -2.2076, -2.1475, -2.4993,  3.0760,  1.3507, -0.4803,  0.0003,
          1.8403, -0.7795, -0.0285, -1.1292, -0.1674,  2.4695, -0.1369,
          1.1456,  3.1679, -0.3965, -1.5465, -1.9542, -1.3117, -0.2730,
         -1.0788, -0.2687, -3.5032,  1.7496, -1.3449, -1.2721, -0.6779,
          0.4330, -2.5400,  0.8411, -0.9258,  0.8932, -2.1141,  3.5134,
         -0.8301,  2.6460,  0.8543, -1.3381,  0.8489, -0.3284, -2.9983,
          0.9811,  2.0668, -0.3651,  0.3961,  4.5547, -4.5038,  0.6558,
          0.0606,  2.5631,  0.9436, -2.7993,  1

预测（92°W，0°）今年3月份海温（真实值为27.26599℃）

In [18]:
y_test = net(X_test) + month_mean[2, 5255]
y_test.item()

25.156715393066406

# GRU

In [48]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

class gru(nn.Module):

    def __init__(self):
        super(gru, self).__init__()
        self.rnn = nn.GRU(
            input_size=100,
            hidden_size=2,
            num_layers=1,
            batch_first=True
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x, _ = self.rnn(x, None)
        x = self.fc(x[:, -1, :])
        return x

In [49]:
net = gru()

criterion = nn.MSELoss()
optimizer = AdaBound(net.parameters(), lr=0.0001)

In [50]:
for epoch in range(40000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        outputs = net(X_valid)
        loss_valid = criterion(outputs, y_valid)
        print(epoch + 1, loss.item(), loss_valid.item())
        
print('Finished Training')

1 1.0471551418304443 1.7867462635040283
1001 0.5708805322647095 1.4725346565246582
2001 0.338470458984375 1.6608353853225708
3001 0.23948559165000916 1.7719521522521973
4001 0.18859748542308807 2.112320899963379
5001 0.16699345409870148 2.272190809249878
6001 0.14843274652957916 2.4154255390167236
7001 0.13495205342769623 2.3713748455047607
8001 0.12770815193653107 2.4031126499176025
9001 0.12243083119392395 2.4529356956481934
10001 0.11725673079490662 2.461758852005005
11001 0.10965229570865631 2.357987880706787
12001 0.10369528084993362 2.158215284347534
13001 0.09754501283168793 2.1412532329559326
14001 0.09261450171470642 1.989659070968628
15001 0.09003999084234238 1.8380005359649658
16001 0.08835628628730774 1.7944215536117554
17001 0.08666938543319702 1.8536418676376343
18001 0.08406150341033936 1.9813849925994873
19001 0.08242233842611313 1.9778618812561035
20001 0.08084326982498169 1.9469146728515625
21001 0.07954158633947372 1.99197256565094
22001 0.0785195454955101 2.03409767

In [43]:
outputs = net(X_valid)
loss = criterion(outputs, y_valid)

for i in range(60):
    print(y_valid[i].item() + month_mean[i % 12, 5255], outputs[i].item() + month_mean[i % 12, 5255])
    
print('\n')

print(loss.item())

25.193620695879584 26.962858601620322
25.725605010986328 27.34829831123352
26.790624619314546 27.675266698787087
26.549350738211682 26.471346705838254
26.679452894863328 25.41476666801854
26.143993345059847 24.371585574902987
24.624321022786592 23.83999363497684
23.667221065571432 22.94352531056655
23.38800243327492 22.765628375505145
23.729892743261235 23.182628524930852
24.111202208619368 23.508448092561018
24.508861539865794 24.12108903935081
25.268852248003608 25.26747266869796
26.084409713745117 25.91914939880371
27.0227508403753 27.020505742022866
27.623594253941587 26.31566992370706
27.091974376377305 25.248362778362473
26.963388410367465 24.13625940398166
26.370748439588045 23.243037560739015
24.83215152840865 22.802320118954306
25.315359153245623 22.764975823854144
25.57662965122022 23.054406893880742
26.050239650826704 25.194422571282637
26.717212734724345 25.718181429411235
27.22409240823043 24.80542854886306
27.63623046875 26.273600816726685
28.250402406642312 26.5535732068

In [44]:
params = net.state_dict()
for key, value in params.items():
    print(key)
    print(value)

rnn.weight_ih_l0
tensor([[ 0.8344, -0.3500, -0.0581,  0.0982, -0.6910, -1.0347,  0.3074,
          0.2870,  0.8660, -0.5590, -0.6626,  0.0441,  0.6067, -0.1940,
         -0.4461,  0.1219, -0.7096, -0.1682,  0.3684, -0.6149, -0.8333,
          0.4449, -0.2600,  0.9455, -0.4028,  0.0580, -0.4838,  0.4083,
         -0.4237,  0.3088, -0.1605, -0.4153,  0.0464,  0.9482, -0.8600,
         -0.6242,  0.2687, -0.2363, -0.4043, -0.8294,  0.1945,  0.6232,
          0.0324, -0.8341, -0.3535,  0.1316,  0.3326,  0.0675,  0.7847,
          0.1195, -0.3454,  0.6425,  0.4323,  0.3332,  0.5363, -0.0327,
         -0.6294, -0.0044, -0.7570, -0.6438, -0.7693,  1.0778,  0.4016,
         -0.6724,  0.6232,  0.3999,  0.1594, -0.2706,  0.0476, -0.2164,
          0.8526, -0.7801,  0.9971, -1.1252, -0.8131, -0.6199, -0.0972,
         -0.6818, -0.6804,  0.6346, -0.4428, -0.3139,  0.5396, -0.6833,
         -0.1957, -0.5859, -0.5580, -1.1487,  0.5200,  0.3138, -0.3225,
         -0.7316,  0.0303,  0.9403,  0.3220, -0